# Experiment 2:

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/CA_Repo')
from gm_utils_v2 import gradient_matching, get_network, get_daparam, evaluate_synset, save_image, match_loss, get_dataset, train_and_test_model
from helpers import plot_gradient_loss_by_iteration_by_class_and_total

Mounted at /content/drive


In [ ]:
class Args:
    method = "DC"
    dataset = "MNIST"
    model =  "ConvNet"
    hessian = "False"
    ipc = 1  # Images per class
    num_exp = 1
    num_eval = 3  # Number of evaluations per experiment
    Iteration = 15  # Training iterations
    lr_img = 0.1  # Learning rate for synthetic images
    lr_net = 0.01  # Learning rate for network
    batch_real = 256  # Batch size for real data
    batch_train = 128  # Batch size for synthetic data training
    init = "noise"  # Initialize synthetic data from noise
    eval_mode = "S"
    data_path = "./data"
    save_path = "./results"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    epoch_eval_train = 25  # Number of epochs to train evaluation models
    outer_loop = 10  # Number of outer-loop updates for synthetic data\
    dis_metric = 'ours'

In [ ]:
args = Args()
args.model = "ConvNet"
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)
eval_it_pool = [0, 10, args.Iteration]

### Random

In [ ]:
args.init = "noise"
args.save_path = "./results/" + args.init

In [ ]:
accs_all_exps, data_save, gradient_losses, class_gradient_losses = gradient_matching(
    args=args,
    dst_train=dst_train,
    num_classes=num_classes,
    im_size=im_size,
    channel=channel,
    std=std,
    mean=mean,
    eval_it_pool = eval_it_pool,
    testloader=testloader,
    get_network=get_network,
    evaluate_synset=evaluate_synset,
    save_image=save_image,
    match_loss=match_loss,
    get_daparam=get_daparam
)

Class distribution:
Class 0: 5923 images
Class 1: 6742 images
Class 2: 5958 images
Class 3: 6131 images
Class 4: 5842 images
Class 5: 5421 images
Class 6: 5918 images
Class 7: 6265 images
Class 8: 5851 images
Class 9: 5949 images
Initializing synthetic data from random noise...
[2024-12-16 21:04:24] Training begins
Iteration# 0
Evaluating model ConvNet at iteration 0...
Image grid saved to ./results/vis_DC_MNIST_ConvNet_1ipc_single_iter0.png
Iteration 0: Total Gradient Loss = 335605.0293
Iteration# 1
Iteration 1: Total Gradient Loss = 59566.6975
Iteration# 2
Iteration 2: Total Gradient Loss = 69155.2642
Iteration# 3
Iteration 3: Total Gradient Loss = 22221.7222
Iteration# 4
Iteration 4: Total Gradient Loss = 19849.5178
Iteration# 5
Iteration 5: Total Gradient Loss = -4627.2131
Iteration# 6
Iteration 6: Total Gradient Loss = 15289.7244
Iteration# 7
Iteration 7: Total Gradient Loss = 12823.2597
Iteration# 8
Iteration 8: Total Gradient Loss = 9997.4252
Iteration# 9
Iteration 9: Total Grad

In [ ]:
accs_all_exps

{'ConvNet': [0.0957,
  0.1177,
  0.146,
  0.6539,
  0.6379,
  0.6511,
  0.6657,
  0.65,
  0.7117]}

In [ ]:
# Train and test on synthetic data
test_loader_real = testloader
print(f"Training {args.model} on the synthetic data")
synthetic_images, synthetic_labels = data_save[0]  # Get the first experiment's synthetic data
synthetic_dataset = TensorDataset(synthetic_images, synthetic_labels)
train_loader_synthetic = DataLoader(synthetic_dataset, batch_size=64, shuffle=True)
synthetic_model = get_network(args.model, channel, num_classes, im_size)
synthetic_accuracy = train_and_test_model(synthetic_model, train_loader_synthetic, test_loader_real, num_epochs=10, device=args.device)
print(f"Accuracy on test set (trained on synthetic data): {synthetic_accuracy:.4f}")


Training ConvNet on the synthetic data
Epoch 1/10, Loss: 2.3314
Epoch 2/10, Loss: 1.8295
Epoch 3/10, Loss: 1.3390
Epoch 4/10, Loss: 1.0443
Epoch 5/10, Loss: 0.7752
Epoch 6/10, Loss: 0.5799
Epoch 7/10, Loss: 0.4154
Epoch 8/10, Loss: 0.2918
Epoch 9/10, Loss: 0.2090
Epoch 10/10, Loss: 0.1498
Accuracy on test set (trained on synthetic data): 0.8256


In [ ]:
plot_gradient_loss_by_iteration_by_class_and_total(class_gradient_losses, gradient_losses, save_path="./results/" + args.init + "/" )

Gradient loss by iteration plot for Class 0 saved to ./results/noise/_class0.png
Gradient loss by iteration plot for Class 1 saved to ./results/noise/_class1.png
Gradient loss by iteration plot for Class 2 saved to ./results/noise/_class2.png
Gradient loss by iteration plot for Class 3 saved to ./results/noise/_class3.png
Gradient loss by iteration plot for Class 4 saved to ./results/noise/_class4.png
Gradient loss by iteration plot for Class 5 saved to ./results/noise/_class5.png
Gradient loss by iteration plot for Class 6 saved to ./results/noise/_class6.png
Gradient loss by iteration plot for Class 7 saved to ./results/noise/_class7.png
Gradient loss by iteration plot for Class 8 saved to ./results/noise/_class8.png
Gradient loss by iteration plot for Class 9 saved to ./results/noise/_class9.png
Total gradient loss by iteration plot saved to ./results/noise/_total.png


### Sample

In [ ]:
args.init = "real"
args.save_path = "./results/" + args.init

In [ ]:
accs_all_exps, data_save, gradient_losses, class_gradient_losses = gradient_matching(
    args=args,
    dst_train=dst_train,
    num_classes=num_classes,
    im_size=im_size,
    channel=channel,
    std=std,
    mean=mean,
    eval_it_pool = eval_it_pool,
    testloader=testloader,
    get_network=get_network,
    evaluate_synset=evaluate_synset,
    save_image=save_image,
    match_loss=match_loss,
    get_daparam=get_daparam
)


Class distribution:
Class 0: 5923 images
Class 1: 6742 images
Class 2: 5958 images
Class 3: 6131 images
Class 4: 5842 images
Class 5: 5421 images
Class 6: 5918 images
Class 7: 6265 images
Class 8: 5851 images
Class 9: 5949 images
Initializing synthetic data from random real images...
[2024-12-16 22:21:34] Training begins
Iteration# 0
Evaluating model ConvNet at iteration 0...
Image grid saved to ./results/real/vis_DC_MNIST_ConvNet_1ipc_single_iter0.png
Iteration 0: Total Gradient Loss = 95469.2314
Iteration# 1
Iteration 1: Total Gradient Loss = 11761.1500
Iteration# 2
Iteration 2: Total Gradient Loss = 19124.1245
Iteration# 3
Iteration 3: Total Gradient Loss = 6545.4536
Iteration# 4
Iteration 4: Total Gradient Loss = 2802.1804
Iteration# 5
Iteration 5: Total Gradient Loss = 20326.4940
Iteration# 6
Iteration 6: Total Gradient Loss = 1355.2267
Iteration# 7
Iteration 7: Total Gradient Loss = 1441.9426
Iteration# 8
Iteration 8: Total Gradient Loss = -10207.0422
Iteration# 9
Iteration 9: To

In [ ]:
accs_all_exps

{'ConvNet': [0.3978,
  0.361,
  0.325,
  0.6367,
  0.6669,
  0.6125,
  0.6758,
  0.6295,
  0.6601]}

In [ ]:
# Train and test on synthetic data
test_loader_real = testloader
print(f"Training {args.model} on the synthetic data")
synthetic_images, synthetic_labels = data_save[0]  # Get the first experiment's synthetic data
synthetic_dataset = TensorDataset(synthetic_images, synthetic_labels)
train_loader_synthetic = DataLoader(synthetic_dataset, batch_size=64, shuffle=True)
synthetic_model = get_network(args.model, channel, num_classes, im_size)
synthetic_accuracy = train_and_test_model(synthetic_model, train_loader_synthetic, test_loader_real, num_epochs=10, device=args.device)
print(f"Accuracy on test set (trained on synthetic data): {synthetic_accuracy:.4f}")


Training ConvNet on the synthetic data
Epoch 1/10, Loss: 2.4022
Epoch 2/10, Loss: 1.8161
Epoch 3/10, Loss: 1.3083
Epoch 4/10, Loss: 0.9897
Epoch 5/10, Loss: 0.7380
Epoch 6/10, Loss: 0.5271
Epoch 7/10, Loss: 0.3766
Epoch 8/10, Loss: 0.2632
Epoch 9/10, Loss: 0.1835
Epoch 10/10, Loss: 0.1309
Accuracy on test set (trained on synthetic data): 0.8082


In [ ]:
plot_gradient_loss_by_iteration_by_class_and_total(class_gradient_losses, gradient_losses, save_path="./results/" + args.init + "_" )

Gradient loss by iteration plot for Class 0 saved to ./results/real__class0.png
Gradient loss by iteration plot for Class 1 saved to ./results/real__class1.png
Gradient loss by iteration plot for Class 2 saved to ./results/real__class2.png
Gradient loss by iteration plot for Class 3 saved to ./results/real__class3.png
Gradient loss by iteration plot for Class 4 saved to ./results/real__class4.png
Gradient loss by iteration plot for Class 5 saved to ./results/real__class5.png
Gradient loss by iteration plot for Class 6 saved to ./results/real__class6.png
Gradient loss by iteration plot for Class 7 saved to ./results/real__class7.png
Gradient loss by iteration plot for Class 8 saved to ./results/real__class8.png
Gradient loss by iteration plot for Class 9 saved to ./results/real__class9.png
Total gradient loss by iteration plot saved to ./results/real__total.png


### Black

In [ ]:
args.init = "black"
args.save_path = "./results/" + args.init

In [ ]:
accs_all_exps, data_save, gradient_losses, class_gradient_losses = gradient_matching(
    args=args,
    dst_train=dst_train,
    num_classes=num_classes,
    im_size=im_size,
    channel=channel,
    std=std,
    mean=mean,
    eval_it_pool = eval_it_pool,
    testloader=testloader,
    get_network=get_network,
    evaluate_synset=evaluate_synset,
    save_image=save_image,
    match_loss=match_loss,
    get_daparam=get_daparam
)


Class distribution:
Class 0: 5923 images
Class 1: 6742 images
Class 2: 5958 images
Class 3: 6131 images
Class 4: 5842 images
Class 5: 5421 images
Class 6: 5918 images
Class 7: 6265 images
Class 8: 5851 images
Class 9: 5949 images
Initializing synthetic data with black images...
[2024-12-16 23:35:53] Training begins
Iteration# 0
Evaluating model ConvNet at iteration 0...
Image grid saved to ./results/black/vis_DC_MNIST_ConvNet_1ipc_single_iter0.png
Iteration 0: Total Gradient Loss = 6267552.8906
Iteration# 1
Iteration 1: Total Gradient Loss = 1645550.0000
Iteration# 2
Iteration 2: Total Gradient Loss = 1781266.8125
Iteration# 3
Iteration 3: Total Gradient Loss = 1645173.9688
Iteration# 4
Iteration 4: Total Gradient Loss = 1822417.0938
Iteration# 5
Iteration 5: Total Gradient Loss = 1752359.4688
Iteration# 6
Iteration 6: Total Gradient Loss = 2084436.2969
Iteration# 7
Iteration 7: Total Gradient Loss = 1725458.1875
Iteration# 8
Iteration 8: Total Gradient Loss = 1415699.7188
Iteration# 9

In [ ]:
accs_all_exps

{'ConvNet': [0.1212,
  0.0753,
  0.0983,
  0.0608,
  0.0454,
  0.0555,
  0.099,
  0.1101,
  0.0972]}

In [ ]:
# Train and test on synthetic data
test_loader_real = testloader
print(f"Training {args.model} on the synthetic data")
synthetic_images, synthetic_labels = data_save[0]  # Get the first experiment's synthetic data
synthetic_dataset = TensorDataset(synthetic_images, synthetic_labels)
train_loader_synthetic = DataLoader(synthetic_dataset, batch_size=64, shuffle=True)
synthetic_model = get_network(args.model, channel, num_classes, im_size)
synthetic_accuracy = train_and_test_model(synthetic_model, train_loader_synthetic, test_loader_real, num_epochs=10, device=args.device)
print(f"Accuracy on test set (trained on synthetic data): {synthetic_accuracy:.4f}")


Training ConvNet on the synthetic data
Epoch 1/10, Loss: 2.3245
Epoch 2/10, Loss: 0.9602
Epoch 3/10, Loss: 0.4392
Epoch 4/10, Loss: 0.2277
Epoch 5/10, Loss: 0.1308
Epoch 6/10, Loss: 0.0803
Epoch 7/10, Loss: 0.0522
Epoch 8/10, Loss: 0.0355
Epoch 9/10, Loss: 0.0253
Epoch 10/10, Loss: 0.0186
Accuracy on test set (trained on synthetic data): 0.0952


In [ ]:
plot_gradient_loss_by_iteration_by_class_and_total(class_gradient_losses, gradient_losses, save_path="./results/" + args.init + "_" )

Gradient loss by iteration plot for Class 0 saved to ./results/black__class0.png
Gradient loss by iteration plot for Class 1 saved to ./results/black__class1.png
Gradient loss by iteration plot for Class 2 saved to ./results/black__class2.png
Gradient loss by iteration plot for Class 3 saved to ./results/black__class3.png
Gradient loss by iteration plot for Class 4 saved to ./results/black__class4.png
Gradient loss by iteration plot for Class 5 saved to ./results/black__class5.png
Gradient loss by iteration plot for Class 6 saved to ./results/black__class6.png
Gradient loss by iteration plot for Class 7 saved to ./results/black__class7.png
Gradient loss by iteration plot for Class 8 saved to ./results/black__class8.png
Gradient loss by iteration plot for Class 9 saved to ./results/black__class9.png
Total gradient loss by iteration plot saved to ./results/black__total.png


### White

In [ ]:
args.init = "white"
args.save_path = "./results/" + args.init

In [ ]:
accs_all_exps, data_save, gradient_losses, class_gradient_losses = gradient_matching(
    args=args,
    dst_train=dst_train,
    num_classes=num_classes,
    im_size=im_size,
    channel=channel,
    std=std,
    mean=mean,
    eval_it_pool = eval_it_pool,
    testloader=testloader,
    get_network=get_network,
    evaluate_synset=evaluate_synset,
    save_image=save_image,
    match_loss=match_loss,
    get_daparam=get_daparam
)


Class distribution:
Class 0: 5923 images
Class 1: 6742 images
Class 2: 5958 images
Class 3: 6131 images
Class 4: 5842 images
Class 5: 5421 images
Class 6: 5918 images
Class 7: 6265 images
Class 8: 5851 images
Class 9: 5949 images
Initializing synthetic data with white images...
[2024-12-17 00:44:12] Training begins
Iteration# 0
Evaluating model ConvNet at iteration 0...
Image grid saved to ./results/white/vis_DC_MNIST_ConvNet_1ipc_single_iter0.png
Iteration 0: Total Gradient Loss = 575427.1738
Iteration# 1
Iteration 1: Total Gradient Loss = 35137.0575
Iteration# 2
Iteration 2: Total Gradient Loss = 45359.7554
Iteration# 3
Iteration 3: Total Gradient Loss = 13899.9748
Iteration# 4
Iteration 4: Total Gradient Loss = 21397.5546
Iteration# 5
Iteration 5: Total Gradient Loss = 45190.6729
Iteration# 6
Iteration 6: Total Gradient Loss = 48162.6880
Iteration# 7
Iteration 7: Total Gradient Loss = 26810.7372
Iteration# 8
Iteration 8: Total Gradient Loss = 26460.2877
Iteration# 9
Iteration 9: Tot

In [ ]:
accs_all_exps

{'ConvNet': [0.1101,
  0.101,
  0.0878,
  0.6664,
  0.6352,
  0.669,
  0.629,
  0.684,
  0.6495]}

In [ ]:
# Train and test on synthetic data
test_loader_real = testloader
print(f"Training {args.model} on the synthetic data")
synthetic_images, synthetic_labels = data_save[0]  # Get the first experiment's synthetic data
synthetic_dataset = TensorDataset(synthetic_images, synthetic_labels)
train_loader_synthetic = DataLoader(synthetic_dataset, batch_size=64, shuffle=True)
synthetic_model = get_network(args.model, channel, num_classes, im_size)
synthetic_accuracy = train_and_test_model(synthetic_model, train_loader_synthetic, test_loader_real, num_epochs=10, device=args.device)
print(f"Accuracy on test set (trained on synthetic data): {synthetic_accuracy:.4f}")


Training ConvNet on the synthetic data
Epoch 1/10, Loss: 2.3635
Epoch 2/10, Loss: 1.7900
Epoch 3/10, Loss: 1.3213
Epoch 4/10, Loss: 0.9779
Epoch 5/10, Loss: 0.7104
Epoch 6/10, Loss: 0.5052
Epoch 7/10, Loss: 0.3569
Epoch 8/10, Loss: 0.2516
Epoch 9/10, Loss: 0.1757
Epoch 10/10, Loss: 0.1232
Accuracy on test set (trained on synthetic data): 0.8027


In [ ]:
plot_gradient_loss_by_iteration_by_class_and_total(class_gradient_losses, gradient_losses, save_path="./results/" + args.init + "_" )

Gradient loss by iteration plot for Class 0 saved to ./results/white__class0.png
Gradient loss by iteration plot for Class 1 saved to ./results/white__class1.png
Gradient loss by iteration plot for Class 2 saved to ./results/white__class2.png
Gradient loss by iteration plot for Class 3 saved to ./results/white__class3.png
Gradient loss by iteration plot for Class 4 saved to ./results/white__class4.png
Gradient loss by iteration plot for Class 5 saved to ./results/white__class5.png
Gradient loss by iteration plot for Class 6 saved to ./results/white__class6.png
Gradient loss by iteration plot for Class 7 saved to ./results/white__class7.png
Gradient loss by iteration plot for Class 8 saved to ./results/white__class8.png
Gradient loss by iteration plot for Class 9 saved to ./results/white__class9.png
Total gradient loss by iteration plot saved to ./results/white__total.png


### Grey

In [ ]:
args.init = "grey"
args.save_path = "./results/" + args.init

In [ ]:
accs_all_exps, data_save, gradient_losses, class_gradient_losses = gradient_matching(
    args=args,
    dst_train=dst_train,
    num_classes=num_classes,
    im_size=im_size,
    channel=channel,
    std=std,
    mean=mean,
    eval_it_pool = eval_it_pool,
    testloader=testloader,
    get_network=get_network,
    evaluate_synset=evaluate_synset,
    save_image=save_image,
    match_loss=match_loss,
    get_daparam=get_daparam
)


Class distribution:
Class 0: 5923 images
Class 1: 6742 images
Class 2: 5958 images
Class 3: 6131 images
Class 4: 5842 images
Class 5: 5421 images
Class 6: 5918 images
Class 7: 6265 images
Class 8: 5851 images
Class 9: 5949 images
Initializing synthetic data with grey images...
[2024-12-17 01:54:50] Training begins
Iteration# 0
Evaluating model ConvNet at iteration 0...
Image grid saved to ./results/grey/vis_DC_MNIST_ConvNet_1ipc_single_iter0.png
Iteration 0: Total Gradient Loss = 584717.9844
Iteration# 1
Iteration 1: Total Gradient Loss = 124843.9575
Iteration# 2
Iteration 2: Total Gradient Loss = 71785.7734
Iteration# 3
Iteration 3: Total Gradient Loss = 60206.8760
Iteration# 4
Iteration 4: Total Gradient Loss = 38720.9131
Iteration# 5
Iteration 5: Total Gradient Loss = 23483.3658
Iteration# 6
Iteration 6: Total Gradient Loss = 9318.3963
Iteration# 7
Iteration 7: Total Gradient Loss = 9297.1028
Iteration# 8
Iteration 8: Total Gradient Loss = 33330.6423
Iteration# 9
Iteration 9: Total 

In [ ]:
accs_all_exps

{'ConvNet': [0.0754,
  0.098,
  0.0254,
  0.6492,
  0.5648,
  0.5944,
  0.6155,
  0.6542,
  0.6543]}

In [ ]:
# Train and test on synthetic data
test_loader_real = testloader
print(f"Training {args.model} on the synthetic data")
synthetic_images, synthetic_labels = data_save[0]  # Get the first experiment's synthetic data
synthetic_dataset = TensorDataset(synthetic_images, synthetic_labels)
train_loader_synthetic = DataLoader(synthetic_dataset, batch_size=64, shuffle=True)
synthetic_model = get_network(args.model, channel, num_classes, im_size)
synthetic_accuracy = train_and_test_model(synthetic_model, train_loader_synthetic, test_loader_real, num_epochs=10, device=args.device)
print(f"Accuracy on test set (trained on synthetic data): {synthetic_accuracy:.4f}")


Training ConvNet on the synthetic data
Epoch 1/10, Loss: 2.3585
Epoch 2/10, Loss: 1.6897
Epoch 3/10, Loss: 1.1926
Epoch 4/10, Loss: 0.8663
Epoch 5/10, Loss: 0.5919
Epoch 6/10, Loss: 0.3965
Epoch 7/10, Loss: 0.2658
Epoch 8/10, Loss: 0.1814
Epoch 9/10, Loss: 0.1276
Epoch 10/10, Loss: 0.0904
Accuracy on test set (trained on synthetic data): 0.7608


In [ ]:
plot_gradient_loss_by_iteration_by_class_and_total(class_gradient_losses, gradient_losses, save_path="./results/" + args.init + "_" )

Gradient loss by iteration plot for Class 0 saved to ./results/grey__class0.png
Gradient loss by iteration plot for Class 1 saved to ./results/grey__class1.png
Gradient loss by iteration plot for Class 2 saved to ./results/grey__class2.png
Gradient loss by iteration plot for Class 3 saved to ./results/grey__class3.png
Gradient loss by iteration plot for Class 4 saved to ./results/grey__class4.png
Gradient loss by iteration plot for Class 5 saved to ./results/grey__class5.png
Gradient loss by iteration plot for Class 6 saved to ./results/grey__class6.png
Gradient loss by iteration plot for Class 7 saved to ./results/grey__class7.png
Gradient loss by iteration plot for Class 8 saved to ./results/grey__class8.png
Gradient loss by iteration plot for Class 9 saved to ./results/grey__class9.png
Total gradient loss by iteration plot saved to ./results/grey__total.png
